### Imports

In [6]:
%matplotlib inline

import numpy as np
import NLPlib as nlp

import csv
import itertools

import re
import HTMLParser

from matplotlib import pyplot as plt

### Part 1: Pre-process, tokenize and tag

In [8]:
tagger = nlp.NLPlib()

unpickle the dictionary
Initialized lexHash from pickled data.


In [10]:
sent = ["tag", "me"]

In [13]:
tags = tagger.tag(sent)
tags

['NN', 'PRP']

CSV Format:

1. the polarity of the tweet (0 = negative emotion, 4 = positive emotion)
2. the id of the tweet (e.g., 2087)
3. the date of the tweet (e.g., Sat May 16 23:58:44 UTC 2009)
4. the query (e.g., lyx). If there is no query, then this value is NO QUERY. 
5. the user that tweeted (e.g., robotickilldozr)
6. the text of the tweet (e.g., Lyx is cool)

In [26]:
GID = 4
class_zero_data = [GID * 5500, GID * 5500 + 10] # (GID + 1) * 5500 - 1]
class_four_data = [GID * 5500 + 800000, GID * 5500 + 800000 + 10] # (GID + 1) * 5500 - 1 + 800000]

In [28]:
with open('training.1600000.processed.noemoticon.csv', 'rb') as train_file:
    reader = csv.reader(train_file)
    
    for row in itertools.islice(reader, *class_zero_data):
        print row
    for row in itertools.islice(reader, *class_four_data):
        print row

['0', '1557393927', 'Sun Apr 19 03:50:43 PDT 2009', 'NO_QUERY', 'davidgarlick', '@Natalia_Bella not much to buy now Woolworth closed down ']
['0', '1557394133', 'Sun Apr 19 03:50:48 PDT 2009', 'NO_QUERY', 'sarah_etf', 'Kill me please -.- ...Oh crap school tommorow ']
['0', '1557394170', 'Sun Apr 19 03:50:48 PDT 2009', 'NO_QUERY', 'hypnotic', "@chriskeating re the labour general secretary meeting with Labour PM's aide - I posted the very same on facebook. BBC gone downhill "]
['0', '1557394563', 'Sun Apr 19 03:50:57 PDT 2009', 'NO_QUERY', 'Robbertt', 'Whole day of homework ahead of name ']
['0', '1557394776', 'Sun Apr 19 03:51:02 PDT 2009', 'NO_QUERY', 'desii____8', "hamlet...romeo n juliet...radio:ACTIVE live at Wembley...McFly tour DVD's too money to me "]
['0', '1557394848', 'Sun Apr 19 03:51:03 PDT 2009', 'NO_QUERY', 'loubeejones', '@charleypearson haha, lucky you. i just got told one!  loubee is not happy!']
['0', '1557395142', 'Sun Apr 19 03:51:09 PDT 2009', 'NO_QUERY', 'musicjunk

In [30]:
row[5]

'working on different politic media plans for the upcoming EU elections... '

##### 1. All html tags and attributes (i.e., /<[^>]+>/) are removed.

In [4]:
def strip_html_tags(tweet):
    regex = re.compile('<[^>]+>')
    return regex.sub('', tweet)

In [5]:
strip_html_tags('<a href="foo.com" class="bar">I Want This <b>text!</b></a>')

'I Want This text!'

##### 2. Html character codes (i.e., &...;) are replaced with an ASCII equivalent.

In [7]:
def replace_html_codes(tweet):
    parser = HTMLParser.HTMLParser()
    return parser.unescape(tweet)

In [12]:
print replace_html_codes('&quot;You win &pound;100 &iexcl; &quot;')

"You win £100 ¡ "


##### 3. All URLs (i.e., tokens beginning with http or www) are removed.

In [26]:
def remove_urls(tweet):
    # Note that this will modify the whitespace when words are separated by
    # more than one space, but that shouldn't matter as we are tokenizing
    # the tweets anyways
    
    return ' '.join(filter(lambda x : not x.startswith(('www', 'http')), tweet.split()))

In [29]:
remove_urls("brad is the best www.youtube.com dancer but not the worst http://www.google.ca singer")

'brad is the best dancer but not the worst singer'

##### 4. The first character in Twitter user names (@) and hash tags (#) are removed.

In [55]:
def remove_hashtags(tweet):
    return ' '.join([ x[1:] if  x.startswith(('@', '#')) else x for x in tweet.split()])

In [57]:
remove_hashtags('brad #donkey @kick face #@other @#test')

'brad donkey kick face @other #test'

### IBM Watson NLP Classifier

{
  "credentials": {
    "url": "https://gateway.watsonplatform.net/natural-language-classifier/api",
    "username": "2bd0e6c7-5784-4967-860c-a9778754fdee",
    "password": "rFs4Solusscl"
  }
}